In [1]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_logger, get_model, get_trainer, init_seed

In [2]:
model='KGAT'
dataset='mind_small_dev'
config_dict = {
    'data_path': 'C:\\Users\\secci\\Workspace\\TFM - Jupyter\\data'
}

In [3]:
config = Config(model=model, dataset=dataset, config_file_list=None, config_dict=config_dict)
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()

logger.info(config)

# dataset filtering
dataset = create_dataset(config)
logger.info(dataset)

# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

# model loading and initialization
model = get_model(config['model'])(config, train_data).to(config['device'])
logger.info(model)

# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(
    train_data, valid_data, saved=saved, show_progress=config['show_progress']
)

# model evaluation
test_result = trainer.evaluate(test_data, load_best_model=saved, show_progress=config['show_progress'])

logger.info('best valid result: {}'.format(best_valid_result))
logger.info('test result: {}'.format(test_result))

return {
    'best_valid_score': best_valid_score,
    'valid_score_bigger': config['valid_metric_bigger'],
    'best_valid_result': best_valid_result,
    'test_result': test_result
}

30 Jan 12:14    INFO General Hyper Parameters: 
gpu_id=0
use_gpu=True
seed=2020
state=INFO
reproducibility=True
data_path=C:\Users\secci\Workspace\TFM - Jupyter\data\mind_small_dev
show_progress=True

Training Hyper Parameters: 
checkpoint_dir=saved
epochs=300
train_batch_size=2048
learner=adam
learning_rate=0.001
training_neg_sample_num=1
training_neg_sample_distribution=uniform
eval_step=1
stopping_step=10

Evaluation Hyper Parameters: 
eval_setting=RO_RS,full
group_by_user=True
split_ratio=[0.8, 0.1, 0.1]
leave_one_num=2
real_time_process=False
metrics=['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk=[10]
valid_metric=MRR@10
eval_batch_size=4096

Dataset Hyper Parameters: 
field_separator=	
seq_separator= 
USER_ID_FIELD=user_id
ITEM_ID_FIELD=item_id
RATING_FIELD=rating
TIME_FIELD=timestamp
seq_len=None
LABEL_FIELD=label
threshold=None
NEG_PREFIX=neg_
load_col={'inter': ['user_id', 'item_id', 'rating', 'timestamp'], 'kg': ['head_id', 'relation_id', 'tail_id'], 'link': ['item_id', '

AssertionError: kg data requires field [head_id]